# metaL

## гомоиконичная система (язык) программирования
### метод EDS: исполняемые структуры данных

**Гомоиконичность** (homoiconic) -- свойство языка программирования, когда любая *программа* на этом языке одновременно *является данными*.


В гомоиконичном языке программист не просто имеет доступ к исходному коду, но сам язык специально предусматривает средства удобной работы с частями программ *во время исполнения* (в рантайме).

То есть, если вы включаете в дистрибутив вашей программы на С++ ее исходный код, вы можете работать с текстом программы как с данными, но только на уровне текстовых файлов, или используя сторонние библиотеки анализа. В самом языке С++ никаких средств чтения, модификации и генерации исходного кода нет.

Обратно, в языке Лисп все программы представляются в виде исполняемых списков -- списки одновременно являются и представлением программы, и обычной структурой данных, для работы с которой язык специально создавался.

*Чтобы использовать достоинства гомоиконичности в ваших программах на любых языках (С++/Java/C#/...), в ваши программы нужно встроить интерпретатор, который будет выполнять некоторую структуру данных как программу, и одновременно предоставлять высокоуровневые средства для ее модификации.*

Совершенно не обязательно, чтобы этот интерпретатор был реализацией какого-либо скриптового языка программирования. 

Скриптовые интерпретаторы, встраиваемые в программы, сейчас широко используются для предоставления пользователю возможности настраивать и расширять функционал программ без перекомпиляции. Самый известный пример -- игровые движки:
* ядро написано на С++ или любом другом языке, генерирующим высокоэффективный машинный код
* саму игру пишут на каком-нибудь скриптовом языке типа Lua, Python, JavaSript,..

Самое интересное что *возможно программирование без использования языка программирования* -- достаточно, чтобы в системе был интерпретатор структур данных: **метод EDS -- Executable Data Structure** (испольняемые структуры данных). Чтобы создать программу в такой системе, нужен любой способ, позволяющий создать в памяти исполняемую структуру данных: это может быть графическая рисовалка, парсер текстового формата, графовая база данных, или код на С++, который заставляет компилятор включить такую структуру в исполняемый файл.

## Метапрограммирование

**Метапрограммирование** -- когда одна *программа пишет/модифицирует другую программу* (или саму себя).

Метапрограммирование это метод, который позволяет вам увеличивать свою эффективность как программиста за счет того, что вы расширяете язык, на котором пишете. Если вы ежедневно пишете очень похожий код, в языках, которые умеют мета (Lisp,Nim), вы можете написать небольшую программку-макрос, которая будет запускаться во время компиляции кода (Nim), и генерировать новый код по шаблону или изменять уже существующий так, как *вам* это нужно. Фактически, вы можете добавлять в язык те возможности, которые нужны именно вам для узкого набора задач.

Чтобы можно было пользоваться метапрограммированием в полном объеме, язык или система программирования обязательно должна быть гомоиконичной. Если вы хотите использовать этот метод с промышленными языками программирования, применение EDS-интерпретатора позволит вам быстро и удобно решать ваши задачи, заплатив за это потерями в скорости работы программ и памятью (см. интерпретатор vs компилятор в машинный код).

## Представление программ

Известные формы представления программ:
* машинный код
* исходный код на некотором языке (текстовые файлы) -- все промышленные языки программирования
* списки -- язык Лисп

`metaL` (*meta* **L**anguage) предлагает еще одну форму:
* объектные графы -- `metaL`

**Граф** -- математическое понятие, и структура данных очень широко используемая в программировании.

Граф состоит из вершин -- узлов, и ребер, которые соединяют узлы между собой. Ориентированный граф -- частный случай графа, в котором для каждого ребра задано направление, и известны начальный и конечный узел.

**Объект** -- элемент данных, который связывает 
* набор полей данных, задающих состояние объекта
* методы для работы с этими данными, и с самим объектом как с единой сущностью

**Объектный граф** -- структура данных, в которой 
* узлами являются объекты, а
* ребра задаются как ссылки из объекта на другие объекты\
Вполне естественно, что объектный граф является направленным, и может быть циклическим (объект ссылается сам на себя)

## Фрейм

Структура данных, очень похожая на объектный граф, описана в книге Марвин Мински **Фреймы для представления знаний**, поэтому будем такие объекты называть **фреймами**.

In [8]:
class Frame:
    def __init__(self,V):
        # метка типа/класса
        self.type = self.__class__.__name__.lower()
        # имя фрейма, или скалярное значение: число, строка,..
        self.val  = V
        # атрибуты = слоты = ассоциативный массив
        self.slot = {}
        # список = стек = очередь = вложенные элементы в AST
        self.nest = []

In [9]:
print( Frame('Hello World') )

**Каждый узел объектного графа -- универсальный контейнер данных.**

* `type` указывает тип фрейма, который в том числе выводится пользователю. Это поле обязательно для языков, которые не поддерживают RTTI: не умеют обределять тип произвольного объекта динамически, при выполнении программы. В Python есть встроенная функция `isinstance()`, но поле появилось из-за требований библиотеки PLY.

* `val` содержит данные, которые отличают один фрейм от другого: имя фрейма, или скалярная величина.
* `slot` хранит именованные ссылки на другие фреймы -- задает ребра объектного графа
* `nest` упорядоченный контейнер данных, которого не было в оригинальных фреймах Мински, но он необходим для разбора текстовых форматов (вложенные элементы дерева разбора), и применения в качестве стека (LIFO) и очереди (FIFO)

Любой узел может хранить любые другие данные, упакованные в дополнительные поля -- достаточно отнаследовать новый класс об базового `Frame`.

### Вывод графа фреймов в виде дерева

Для работы с деревом фреймов необходимо средство просмотра полученного объектного графа. Как вы видете выше, вывод `print()` совершенно неинформативен, из него нельзя понять не только состав фрейма, но даже просто его имя. Как минимум, нам нужен способ получать текстовый дамп, и еще желательно иметь способ просмотра графа в графике.

In [14]:
class Frame(Frame): # финт для Jupyter Notebook: наследование классом самого себя
    # системный метод: конвертация в строку для print() и выражений типа '%s' % frame
    def __repr__(self):
        return self.dump()
    # полный дамп фреймового графа в виде дерева
    def dump(self,depth=0,prefix=''):
        # сначала только заголовок, оттабулированный на глубину вложения фрейма
        tree = self._pad(depth) + self.head(prefix)
        # выводим слоты в формате <имя> = <фрейм на который ссылаемся>
        for i in self.slot:
            tree += self.slot[i].dump(depth+1,prefix=i+' = ')
        # вложенные элементы
        for j in self.nest:
            tree += j.dump(depth+1)
        return tree
    # дамп только заголовока фрейма
    def head(self,prefix=''):
        return '%s<%s:%s> @%x' % (prefix,self.type,self._val(),id(self))
    # отбивка на глубину вложения дерева дампа
    def _pad(self,depth):
        return '\n' + '\t' * depth
    # .val может быть произвольным нестроковым типом,
    # поэтому нужна индивидуальное преобразование в строку для разных типов чисел,..
    def _val(self):
        return str(self.val)

In [15]:
Frame('Hello World')


<frame:Hello World> @7f15e8180978

Для пустого элементарного фрейма выводится только заголовок, начинающийся с пустой строки:
* префикс был пустым, поэтому в начале не добавляются никакие дополнительные символы
* `<тип:` метка типа позволяет отличить такие объекты как `<integer:123>` и `<string:123>`
* `:строка>` значение объекта, строка -- самый универсальный тип, который может хранить любые данные
* `@DeadBeef` уникальный (шестнадцатеричный) идентификатор объекта в Python, если вы создадите два фрейма с одинаковым значением, вы все равно сможете их отличить (например объект-оригинал, и его копию)

In [17]:
Frame(123),Frame('123') # .val назначаются разные типы, на дампе выглядит одинаково

(
 <frame:123> @7f15e81804e0, 
 <frame:123> @7f15e8180b00)

In [19]:
Frame(123),Frame(123) # два разных фрейма с одинаковым значением, отличаются по id()

(
 <frame:123> @7f15e8180b70, 
 <frame:123> @7f15e813f128)